In [24]:
import math
import torch
import numpy as np
import gpytorch

In [25]:
n = 20
train_x = torch.zeros(pow(n, 2), 2)
for i in range(n):
    for j in range(n):
        # Each coordinate varies from 0 to 1 in n=100 steps
        train_x[i * n + j][0] = float(i) / (n - 1)
        train_x[i * n + j][1] = float(j) / (n - 1)

train_y_1 = (
    torch.sin(train_x[:, 0])
    + torch.cos(train_x[:, 1]) * (2 * math.pi)
    + torch.randn_like(train_x[:, 0]).mul(0.01)
) / 4
train_y_2 = (
    torch.sin(train_x[:, 0])
    + torch.cos(train_x[:, 1]) * (2 * math.pi)
    + torch.randn_like(train_x[:, 0]).mul(0.01)
)

train_y = torch.stack([train_y_1, train_y_2], -1)

test_x = torch.rand((n, len(train_x.shape)))
test_y_1 = (
    torch.sin(test_x[:, 0])
    + torch.cos(test_x[:, 1]) * (2 * math.pi)
    + torch.randn_like(test_x[:, 0]).mul(0.01)
) / 4
test_y_2 = (
    torch.sin(test_x[:, 0])
    + torch.cos(test_x[:, 1]) * (2 * math.pi)
    + torch.randn_like(test_x[:, 0]).mul(0.01)
)
test_y = torch.stack([test_y_1, test_y_2], -1)

In [26]:
train_x = train_x.unsqueeze(0).repeat(2, 1, 1)
train_y = train_y.transpose(-2, -1)

In [27]:
train_y.shape

torch.Size([2, 400])

In [28]:
torch.manual_seed(2)  # For a more robust comparison


class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.MaternKernel(nu=2.5)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


likelihood = gpytorch.likelihoods.GaussianLikelihood(num_tasks=2)
model = MultitaskGPModel(train_x, train_y, likelihood)

model.cuda()
likelihood.cuda()

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(
    [
        {"params": model.parameters()},  # Includes GaussianLikelihood parameters
    ],
    lr=0.1,
)


# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

n_iter = 50
for i in range(n_iter):
    optimizer.zero_grad()
    output = model(train_x)
    loss = -mll(output, train_y).sum()
    loss.backward()
    print("Iter %d/%d - Loss: %.3f" % (i + 1, n_iter, loss.item()))
    optimizer.step()

# Set into eval mode
model.eval()
likelihood.eval()

# Make predictions
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    predictions = likelihood(model(test_x))
    mean = predictions.mean
    lower, upper = predictions.confidence_region()

test_results_gpytorch = np.median(
    (test_y.transpose(-2, -1) - mean) / test_y.transpose(-2, -1), axis=1
)
print(test_results_gpytorch)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument other in method wrapper_CUDA__equal)

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, Matern

kernel = (
    1.0 * Matern(length_scale=0.1, length_scale_bounds=(1e-5, 1e5), nu=2.5)
    + WhiteKernel()
)
gp = GaussianProcessRegressor(kernel=kernel, alpha=0.0).fit(
    train_x[0].numpy(), train_y.transpose(-2, -1).numpy()
)
# x_interpolation = test_x.detach().numpy()[np.newaxis, :].transpose()
y_mean_interpol, y_std_norm = gp.predict(test_x.numpy(), return_std=True)

test_results_scitlearn = np.median((test_y.numpy() - y_mean_interpol), axis=0)

/home/alujan/mambaforge-pypy3/envs/multinterp-dev/lib/python3.10/site-packages/sklearn/gaussian_process/_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


In [ ]:
comparisson = (test_results_scitlearn - test_results_gpytorch) / test_results_scitlearn
print(
    "Variable 1: scitkit learn is more accurate my factor: " + str(abs(comparisson[0]))
)
print("Variable 2: scitkit learn is more accurate my factor: " + str(comparisson[1]))

Variable 1: scitkit learn is more accurate my factor: 7.576246355781184
Variable 2: scitkit learn is more accurate my factor: 1.04243016558855


In [ ]:
print(test_results_scitlearn)

[-7.95036322e-05 -2.05220096e-03]
